In [1]:
import sys
import os

In [2]:
sys.path.append('/app/darkflow/')
sys.path.append('/app/')
sys.path

['',
 '/usr/local/lib/python36.zip',
 '/usr/local/lib/python3.6',
 '/usr/local/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/site-packages/IPython/extensions',
 '/root/.ipython',
 '/app/darkflow/',
 '/app/']

In [3]:
import yolo_darkflow as yolo
from main import *

In [4]:
yoloDetector = yolo.getYoloDetector(threshold = 0.1)

Parsing /app/darkflow/cfg/yolo.cfg
Loading /app/darkflow/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.05167865753173828s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep

In [17]:
#import requests

#r = requests.get('http://www.clicrbs.com.br/rbs/image/23076410.jpg?w=600')#'http://www.imagens.usp.br/wp-content/uploads/28112013transitofotomarcossantos001.jpg')
#img = r.raw.read()

In [18]:
#r.content

In [19]:
import cv2
image = cv2.imread('/app/images.jpeg')
image.shape

(183, 275, 3)

In [20]:
frame = image
peopleLastDetected = []
detectionAreas = readAreas(frame)

In [21]:
yoloDetections = yoloDetector.return_predict(frame)
people = yoloDetections2people(yoloDetections)
trackedPeople = tracker(peopleLastDetected, people)
peopleLastDetected = trackedPeople
areaCounts = countPeopleInEachArea(detectionAreas,trackedPeople)

In [22]:
peopleLastDetected

[<main.Person at 0x7f04bede3da0>, <main.Person at 0x7f04bede3978>]

In [ ]:
# areaCounts = [1]

In [ ]:
#font = cv2.FONT_HERSHEY_SIMPLEX
#for areaIdx,areaCount in enumerate(areaCounts):
#    cv2.putText(frame,"Area " + str(areaIdx)+" : " +str(areaCount),(10,30+areaIdx*25), 
#        font, 0.8,(0,255,0),2,cv2.LINE_AA)

##recordTimeSpent(trackedPeople,detectionAreas,fps,0)
#frame = drawAreas(frame,detectionAreas)
#visualizeDetections(frame, trackedPeople)


In [23]:
yoloDetections

[{'bottomright': {'x': 40, 'y': 87},
  'confidence': 0.30405664,
  'label': 'person',
  'topleft': {'x': 32, 'y': 71}},
 {'bottomright': {'x': 120, 'y': 158},
  'confidence': 0.82946384,
  'label': 'person',
  'topleft': {'x': 77, 'y': 69}},
 {'bottomright': {'x': 175, 'y': 160},
  'confidence': 0.73966813,
  'label': 'person',
  'topleft': {'x': 136, 'y': 74}},
 {'bottomright': {'x': 152, 'y': 125},
  'confidence': 0.14679013,
  'label': 'car',
  'topleft': {'x': 118, 'y': 114}},
 {'bottomright': {'x': 132, 'y': 60},
  'confidence': 0.45300642,
  'label': 'traffic light',
  'topleft': {'x': 122, 'y': 45}},
 {'bottomright': {'x': 45, 'y': 87},
  'confidence': 0.13367052,
  'label': 'horse',
  'topleft': {'x': 31, 'y': 70}},
 {'bottomright': {'x': 117, 'y': 103},
  'confidence': 0.107185796,
  'label': 'backpack',
  'topleft': {'x': 99, 'y': 81}}]

In [28]:
yoloDetector.graph

In [37]:
import tensorflow as tf
from IPython.display import HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes(size) #"<stripped %d bytes>" % size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [38]:
show_graph(yoloDetector.graph)

MemoryError: 